In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

The COVID-19 pandemic has posed many challenges to the Healthcare companies, for example, shifting consumer demands towards fast and convenient services. As a result, they are forced to take on digital transformations. On the other hand, big tech companies are also looking to occupy the niche markets in the Healthcare industry. Therefore, many companies in those two sectors formed partnetships. 

In this project, we hope to study the relationship between the two sectors by investigating their stock prices.

In [ ]:
GOOGL = pd.read_csv('GOOGL.csv')
AMZN = pd.read_csv('AMZN.csv')
AAPL = pd.read_csv('AAPL.csv')
META = pd.read_csv('META.csv')
MSFT = pd.read_csv('MSFT.csv')

it_df = [GOOGL, AMZN, AAPL, META, MSFT]
for it in it_df:
    print(it.shape)

In [5]:
CVS = pd.read_csv('CVS.csv') # CVS Health Corp.
UNH = pd.read_csv('UNH.csv') # UnitedHealth Group Inc.
MCK = pd.read_csv('MCK.csv') # McKesson Corp. 
ABC = pd.read_csv('ABC.csv')
CI = pd.read_csv('CI.csv') 

#healthcare_df = [CVS[0], UNH[0], MCK[0], ABC[0], CI[0]]

ValueError: Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.

### Model 1. Linear Regression

In this model, we would use the prices of other 9 stocks to predict the 

In [6]:
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create a Linear Regression object
linreg = LinearRegression()

In [ ]:
X = pd.concat([AMZN["4. close"], AAPL["4. close"], 
               META["4. close"], MSFT["4. close"], 
               CVS["4. close"], UNH["4. close"], 
               MCK["4. close"], ABC["4. close"], 
               CI["4. close"]], axis=1)
y = GOOGL["4. close"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

linreg.fit(X_train, y_train)

print('Intercept \t: b = ', linreg.intercept_)
print('Coefficients \t: a = ', linreg.coef_)

In [ ]:
# Predict Response values corresponding to Predictors
y_train_pred = linreg.predict(X_train)
y_test_pred = linreg.predict(X_test)

# Plot the Predictions vs the True values
f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(y_train, y_train_pred, color = "blue")
axes[0].plot(y_train, y_train, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable (Train)")
axes[0].set_ylabel("Predicted values of the Response Variable (Train)")
axes[1].scatter(y_test, y_test_pred, color = "green")
axes[1].plot(y_test, y_test, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable (Test)")
axes[1].set_ylabel("Predicted values of the Response Variable (Test)")
plt.show()

#### Goodness of Fit of the Linear Regression model

In [ ]:
# Check the Goodness of Fit (on Train Data)
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_train, y_train_pred))
print()

# Check the Goodness of Fit (on Test Data)
print("Goodness of Fit of Model \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print()

### Model 2. Moving Average

Moving Average is a very simple model for Time Series modelling.

In [ ]:
def plot_moving_average(series, window, plot_intervals=False, scale=1.96):

    rolling_mean = series.rolling(window=window).mean()
    
    plt.figure(figsize=(17,8))
    plt.title('Moving average\n window size = {}'.format(window))
    plt.plot(rolling_mean, 'g', label='Rolling mean trend')
    
    #Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bound = rolling_mean - (mae + scale * deviation)
        upper_bound = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bound, 'r--', label='Upper bound / Lower bound')
        plt.plot(lower_bound, 'r--')
            
    plt.plot(series[window:], label='Actual values')
    plt.legend(loc='best')
    plt.grid(True)

In [ ]:
#Smooth by the previous 5 days (by week)
plot_moving_average(GOOGL['4. close'], 5)

In [ ]:
#Smooth by the previous month (30 days)
plot_moving_average(GOOGL['4. close'], 30)

In [ ]:
#Smooth by previous quarter (90 days)
plot_moving_average(GOOGL['4. close'], 90, plot_intervals=True)

### Model 2. Exponential Smoothing

Exponential smoothing is similar to Moving Average 

In [ ]:
def exponential_smoothing(series, alpha):

    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

In [ ]:
def plot_exponential_smoothing(series, alphas):
 
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        plt.plot(exponential_smoothing(series, alpha), label="Alpha {}".format(alpha))
    plt.plot(series.values, "c", label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Exponential Smoothing")
    plt.grid(True);

In [ ]:
plot_exponential_smoothing(GOOGL['4. close'], [0.05, 0.3])

### Model 3. Double Exponential Smoothing

In [ ]:
def double_exponential_smoothing(series, alpha, beta):

    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha * value + (1 - alpha) * (level + trend)
        trend = beta * (level - last_level) + (1 - beta) * trend
        result.append(level + trend)
    return result


In [ ]:
def plot_double_exponential_smoothing(series, alphas, betas):
     
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        for beta in betas:
            plt.plot(double_exponential_smoothing(series, alpha, beta), label="Alpha {}, beta {}".format(alpha, beta))
    plt.plot(series.values, label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Double Exponential Smoothing")
    plt.grid(True)

In [ ]:
plot_double_exponential_smoothing(GOOGL['4. close'], alphas=[0.9, 0.02], betas=[0.9, 0.02])

### Model 5. Seasonal autoregressive integraded moving average model (SARIMA)

SARIMA is able to model . It is actually the combination of several component models which are introduced breifly below.

1. Autoregressive model **AR (p)**

2. 

#### Stationarity

A time series is said to be stationary if its statistical properties (constant mean, variance) do not change over time.

A stationary time series is ideal because it is easier to model. However, stock price is frequently not stationary as we often see a growing/declining trend or its variance is changing under different economic contexts.

Dickey-Fuller test is often used to test if a time series is stationary. The null hypothesis of the test is that there exists a unit root.

In [ ]:
from tqdm import tqdm_notebook
from itertools import product
import statsmodels.api as sm
import warnings                                  # do not disturbe mode
warnings.filterwarnings('ignore')

In [ ]:
#Set initial values and some bounds
ps = range(0, 5)
d = 1
qs = range(0, 5)
Ps = range(0, 5)
D = 1
Qs = range(0, 5)
s = 5

#Create a list with all possible combinations of parameters
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
def optimize_SARIMA(parameters_list, d, D, s):
    """
        Return dataframe with parameters and corresponding AIC
        
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order
        D - seasonal integration order
        s - length of season
    """
    
    results = []
    best_aic = float('inf')
    
    for param in tqdm_notebook(parameters_list):
        try: model = sm.tsa.statespace.SARIMAX(GOOGL['4. close'], order=(param[0], d, param[1]),
                                               seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
            
        aic = model.aic
        
        #Save best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])
        
    result_table = pd.DataFrame(results)
#I deleted a few lines here from the original codes so error does not occur when running
    
    return result_table

result_table = optimize_SARIMA(parameters_list, d, D, s)

In [ ]:
#Set parameters that give the lowest AIC (Akaike Information Criteria)
print(result_table)
p, q, P, Q = result_table.parameters[0]

best_model = sm.tsa.statespace.SARIMAX(data.CLOSE, order=(p, d, q),
                                       seasonal_order=(P, D, Q, s)).fit(disp=-1)

print(best_model.summary())